In [1]:
from kLLMmeans import kLLMmeans, get_embeddings, summarize_cluster, sequentialMiniBatchKmeans, miniBatchKLLMeans
from experiment_utils import load_dataset, cluster_metrics, avg_closest_distance
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.preprocessing import LabelEncoder

import numpy as np
import json, pickle

import warnings
warnings.filterwarnings("ignore")

In [4]:
max_iter = 120
emb_type = 'openai'
max_batch_size = 1000

prompt = "The following is a cluster of questions from the AI community. Write a single question that represents the cluster of questions:"
text_type = "Question:"

for year in [2020,2021,2022,2023]:

    with open("processed_data/data_stackexchange_openai_" + str(year) + ".pkl", "rb") as f:
        data_dict = pickle.load(f) 

    try:
        with open("results/sims_stackexchange_results_AI_q_" + str(year) + ".pkl", "rb") as f:
            results_dict = pickle.load(f)
        print('Old results_dict loaded')
    except:
        print('No previous results')
        results_dict = {}

    data_dict['data']['embeddings'] = list(data_dict['embeddings'])
    data_dict['data'] = data_dict['data'][data_dict['data']['Label']=='ai'].copy()
    
    data_dict['data'] = data_dict['data'].sort_values('CreationDate')
    #data_dict['data'] = data_dict['data'][data_dict['data']['Label'].isin(selected_groups)]
    
    text_data = list(data_dict['data']['Text'])

    num_clusters = 10
    
    print(num_clusters)
    
    text_features = list(data_dict['data']['embeddings'])

    for seed in range(1):
        if results_dict.get(seed) is None:
            results_dict[seed] = {}

        #kLLMmeans
        for force_context_length in [50]:
            if results_dict[seed].get(force_context_length) is None:
                results_dict[seed][force_context_length] = {}
                
            for max_llm_iter in [3]:

                if results_dict[seed][force_context_length].get(max_llm_iter) is None:
                    print([year, seed, force_context_length, max_llm_iter])
                    summaries, centroids = miniBatchKLLMeans(text_data, 
                                                            num_clusters,
                                                            max_batch_size = max_batch_size, 
                                                            init = 'k-means++',
                                                            prompt = prompt, text_type = text_type,
                                                            force_context_length = force_context_length, max_llm_iter = max_llm_iter, 
                                                            max_iter = max_iter, tol=1e-4, random_state = seed, 
                                                            emb_type = 'openai', text_features = text_features,
                                                            final_iter = True,
                                                            initial_iter = True)
                    kmeans2 = KMeans(n_clusters=num_clusters, init=centroids, max_iter=1)
                    cluster_assignments = kmeans2.fit_predict(text_features)
                    
                    data_results ={'assignments':cluster_assignments,
                                   'summaries':summaries,
                                   'final_centroids':centroids}
                    
                    
                    print(summaries[-1])
                    results_dict[seed][force_context_length][max_llm_iter] = data_results

                    # Save as pkl file
                    with open("results/sims_stackexchange_results_AI_q_" + str(year) + ".pkl", "wb") as f:
                        pickle.dump(results_dict, f)
                        
                else:
                    data_results = results_dict[seed][force_context_length][max_llm_iter]
                    print([year, seed, force_context_length, max_llm_iter])
                    print(data_dict['summaries'])

                

No previous results
10
[2020, 0, 50, 3]


  File "C:\Users\jairo\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "C:\Users\jairo\anaconda3\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\jairo\anaconda3\Lib\subprocess.py", line 1026, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "C:\Users\jairo\anaconda3\Lib\subprocess.py", line 1538, in _execute_child
    hp, ht, pid, tid = _winapi.CreateProcess(executable, args,
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
Iterating KMeans: 100%|█████████████████████████████████████████████████████████████████| 3/3 [05:42<00:00, 114.17s/it]


['How do different components and techniques in neural networks, such as activation functions, loss functions, weight initialization, and regularization, contribute to the training process and affect the performance and convergence of the model?', 'How can neural networks, particularly transformers and other attention-based models, be effectively utilized and adapted for various natural language processing tasks, such as text generation, translation, and simplification, while addressing challenges like handling variable input/output lengths, incorporating prior knowledge through embeddings, and ensuring efficient training and inference?', 'How can one mathematically define afterstate value functions, including their Bellman equation, and understand their relationship to state and state-action value functions, while also addressing challenges such as local optima in reinforcement learning, exploration strategies, and the use of different divergence measures for policy comparison?', 'How

Iterating KMeans: 100%|█████████████████████████████████████████████████████████████████| 3/3 [06:06<00:00, 122.19s/it]


['How can I optimize the training process of a neural network, considering factors like gradient clipping, batch size, and hyperparameter tuning, while addressing issues such as overfitting, model complexity, and data augmentation to improve model performance and generalization across different datasets and conditions?', 'How can I effectively utilize AI techniques, such as convolutional neural networks (CNNs) and object detection models, for tasks involving image data, including detecting specific objects or features, handling imbalanced datasets, and improving model performance through techniques like transfer learning, data augmentation, and specialized loss functions?', 'What are the best practices and approaches for handling various machine learning challenges, such as dealing with small datasets, feature importance, class imbalance, model interpretability, and dataset splitting, while considering different types of data and tasks like time series prediction, clustering, and class

Iterating KMeans: 100%|█████████████████████████████████████████████████████████████████| 3/3 [06:13<00:00, 124.61s/it]


['How can I address issues with training stability, loss function selection, and gradient problems in neural networks, while ensuring proper initialization, normalization, and understanding of activation functions and their derivatives?', 'How can I effectively preprocess and encode various types of data (e.g., time series, categorical, quantitative) for input into neural networks, and what are the best practices for handling different data types, such as embedding categorical variables, normalizing quantitative data, and dealing with high-dimensional or complex data structures like tensors or graphs?', 'How do language models like ChatGPT understand and process context, commands, and novel instructions without explicit programming logic, and how can they be fine-tuned or adapted for specific tasks such as question answering with multiple contexts or maintaining topic relevance in generated responses?', 'How can reinforcement learning algorithms be adapted or optimized for environments

Iterating KMeans: 100%|█████████████████████████████████████████████████████████████████| 3/3 [06:25<00:00, 128.51s/it]

['How can the Dirac delta function be understood as a special case of a probability density distribution with zero variance, and how does this relate to probabilistic reasoning over datasets, particularly in the context of reinforcement learning, diffusion models, and other machine learning frameworks?', 'How can beginners in machine learning understand the differences between key concepts like models, algorithms, and hypotheses, while also exploring the potential and limitations of AI in solving complex problems, such as the P-NP problem, and addressing challenges like adversarial attacks, hallucinations, and ethical considerations in AI applications?', 'How can I effectively modify and optimize machine learning models, such as WGAN-gp for sparse data or LSTM for time series, while addressing issues like training stability, hyperparameter tuning, and resource management, to improve model performance and scalability across different tasks and datasets?', 'Why do transformer models ofte